In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')
from unidecode import unidecode
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from gensim.models import Word2Vec
from string import punctuation
from sklearn.metrics.pairwise import cosine_similarity
import pickle
import json
import requests


In [2]:
credits=pd.read_csv(r"C:\Users\kannu\OneDrive\Desktop\DataScience\Projects\Movie_Recommender_System\datasets\tmdb_5000_credits.csv")
movies_data=pd.read_csv(r"C:\Users\kannu\OneDrive\Desktop\DataScience\Projects\Movie_Recommender_System\datasets\tmdb_5000_movies.csv")

In [3]:
credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [4]:
movies_data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


### Merging both

In [5]:
combined=movies_data.merge(credits,on='title')

In [6]:
movies_data.shape

(4803, 20)

In [7]:
credits.shape

(4803, 4)

In [8]:
combined.shape

(4809, 23)

### Selecting only required features

In [9]:
combined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4809 entries, 0 to 4808
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4809 non-null   int64  
 1   genres                4809 non-null   object 
 2   homepage              1713 non-null   object 
 3   id                    4809 non-null   int64  
 4   keywords              4809 non-null   object 
 5   original_language     4809 non-null   object 
 6   original_title        4809 non-null   object 
 7   overview              4806 non-null   object 
 8   popularity            4809 non-null   float64
 9   production_companies  4809 non-null   object 
 10  production_countries  4809 non-null   object 
 11  release_date          4808 non-null   object 
 12  revenue               4809 non-null   int64  
 13  runtime               4807 non-null   float64
 14  spoken_languages      4809 non-null   object 
 15  status               

In [10]:
movies=combined[['id','title','overview','genres','keywords','cast','crew']]

In [11]:
movies.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...","[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...","[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...","[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...","[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...","[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...","[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"John Carter is a war-weary, former military ca...","[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","[{""id"": 818, ""name"": ""based on novel""}, {""id"":...","[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [12]:
movies.isna().sum()

id          0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [13]:
movies.dropna(inplace=True)

In [14]:
#from genre we need only name
#from keywords also we need only name
#from cast we will pick top 3 character actual name
#from crew we will pick director


In [15]:
def name_extracter(data):
    list_dictionary=json.loads(data)
    
    
    name_list=[]
    for dictionary in list_dictionary:
        name_list.append(dictionary['name'].replace(" ",""))
    
    return " ".join(name_list)

movies['keywords']=movies['keywords'].apply(name_extracter)


In [16]:
movies['genres']=movies['genres'].apply(name_extracter)

In [17]:
def top_3_actors(data):
    List_dict=json.loads(data)
    count=0
    name=[]
    for dictionary in List_dict:
        if count < 3:
            name.append(dictionary['name'].replace(" ",""))
            count +=1
        else:
            break
    return " ".join(name)
            

In [18]:
movies['cast']=movies['cast'].apply(top_3_actors)

In [19]:
def director(data):
    list_dict=json.loads(data)
    
    for dic in list_dict:
        if dic['job'].lower().__contains__("director"):
            return dic['name'].replace(" ","")
        
            


In [20]:
movies['crew']=movies['crew'].apply(director)

In [21]:
movies

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,SamWorthington ZoeSaldana SigourneyWeaver,JamesCameron
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drugabuse exoticisland eastindiatradingc...,JohnnyDepp OrlandoBloom KeiraKnightley,DariuszWolski
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy basedonnovel secretagent sequel mi6 britis...,DanielCraig ChristophWaltz LéaSeydoux,SamMendes
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dccomics crimefighter terrorist secretidentity...,ChristianBale MichaelCaine GaryOldman,ChristopherNolan
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure ScienceFiction,basedonnovel mars medallion spacetravel prince...,TaylorKitsch LynnCollins SamanthaMorton,AndrewStanton
...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,Action Crime Thriller,unitedstates–mexicobarrier legs arms paperknif...,CarlosGallardo JaimedeHoyos PeterMarquardt,RobertRodriguez
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,Comedy Romance,,EdwardBurns KerryBishé MarshaDietlein,EdwardBurns
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TVMovie,date loveatfirstsight narration investigation ...,EricMabius KristinBooth CrystalLowe,AdamSliwinski
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,,,DanielHenney ElizaCoupe BillPaxton,DanielHsia


In [22]:
movies["tags"]=movies["overview"]+" "+movies["genres"]+" "+movies["keywords"]+" "+movies["cast"]+" "+movies["crew"]

In [23]:
stopwords_list=stopwords.words('english')

In [24]:
#Accented character handing

def accented(data):
    return unidecode(str(data))

def white_space(data):
    import re
    return re.sub(r"\s+"," ",data)

movies_accented=movies['tags'].apply(accented)
movies_white_space=movies_accented.apply(white_space)

In [25]:
#cleaning
def clean(data):
    res=[word.lower() for word in word_tokenize(data) if (word not in punctuation) and (word.lower() not in stopwords_list)]
    return " ".join(res)

movies_cleaned=movies_white_space.apply(clean)

In [26]:
#Performing Lemmatization

def stem(data):
    words=[]
    lemma=WordNetLemmatizer()
    for word in word_tokenize(data):
        words.append(lemma.lemmatize(word,"v"))
    return " ".join(words)

movies_stem=movies_cleaned.apply(stem)

In [27]:
movies_stem

0       22nd century paraplegic marine dispatch moon p...
1       captain barbossa long believe dead come back l...
2       cryptic message bond 's past send trail uncove...
3       follow death district attorney harvey dent bat...
4       john carter war-weary former military captain ...
                              ...                        
4804    el mariachi want play guitar carry family trad...
4805    newlywed couple 's honeymoon upend arrivals re...
4806    `` sign seal deliver `` introduce dedicate qua...
4807    ambitious new york attorney sam send shanghai ...
4808    ever since second grade first saw e.t extrater...
Name: tags, Length: 4806, dtype: object

In [28]:
#Using Count Vectorizer for word embedding

In [29]:
cv=CountVectorizer(max_features=2000,max_df=.95)
cv_data=cv.fit_transform(movies_stem).A

In [30]:
cv_data.shape

(4806, 2000)

In [31]:
#Using TFIDF for word embedding
tfidf=TfidfVectorizer(max_features=2000,max_df=.95)
tfidf_data=tfidf.fit_transform(movies_stem).A

In [32]:
tfidf_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [33]:
# Using Cosine similarity to find top 5 similar movies

In [34]:
cv_similar=cosine_similarity(cv_data)

In [35]:
#cousine similarity with tfidf

tfidf_similar=cosine_similarity(tfidf_data)

In [36]:
cv_similar.shape

(4806, 4806)

In [37]:
#means each vectors (movies) similarity with each one
#At index 0, 0 index movie similary with other movies in each col
# suppose we have to find 0 index movies 5 similar movies, for that we will short and will extract top 5 having most values.

In [38]:
#Recoomender using CountVectorizer

def recommender_cv(movie_title):
    index=movies[movies["title"]==movie_title].index[0]
    similarity=list(enumerate(cv_similar[index]))
    similar_5=sorted(similarity,reverse=True,key=lambda x:x[1])[1:6]
    for data in similar_5:
        print(movies['title'][data[0]])
          


    

In [39]:
#Recommender using tfidf
def recommender_tfidf(movie_title):
    index=movies[movies["title"]==movie_title].index[0]
    similarity=list(enumerate(tfidf_similar[index]))
    similar_5=sorted(similarity,reverse=True,key=lambda x:x[1])[1:6]
    for data in similar_5:
        print(movies['title'][data[0]])
          

### Using Word2Vec for word embedding

In [40]:

word2vec_data=movies_stem.apply(lambda x:x.split()).to_list()


In [41]:
model=Word2Vec(window=20,min_count=2)
model.build_vocab(word2vec_data)

In [42]:
model.corpus_count

4806

In [43]:
model.train(word2vec_data,total_examples=model.corpus_count,epochs=10)

(1883496, 2145800)

In [44]:
# Making function to get document vector

In [45]:
def word2vec(docs):
    all_doc_vector=[]
    for doc in docs:
        doc_vector=[]
        for word in doc:
            
            if word in model.wv:
                doc_vector.append(model.wv[word])
                
        if doc_vector:
            doc_array=np.array(doc_vector)
            avg=doc_array.mean(axis=0)
            all_doc_vector.append(avg)
        else:
            all_doc_vector.append(np.zeros(model.vector_size))
    return all_doc_vector

In [46]:
word2vec_embedding=word2vec(word2vec_data)

In [47]:
len(word2vec_embedding)

4806

In [48]:
word2vec_embedding

[array([-0.42378658,  0.36330906,  0.23888227, -0.10959694,  0.21351187,
        -0.15274832,  0.28410208,  0.22976139, -0.6128194 ,  0.4274747 ,
        -0.35684764,  0.045431  , -0.28811833, -0.40643257,  0.23152922,
         0.11312281, -0.33358616, -0.81784403,  0.09421828, -0.66256046,
         0.36160693,  0.13229723,  0.741559  , -0.29969302, -0.536684  ,
         0.33460924, -0.01992324, -0.39277005, -0.12803277, -0.23103292,
         0.8768541 , -0.42875016, -0.1675526 , -0.10323454,  0.4118387 ,
         0.1513426 ,  0.3103268 , -0.03982326, -0.22594263, -0.3016077 ,
         0.27948332, -0.53472257, -0.18072148, -0.08851387, -0.25598007,
        -0.918083  ,  0.00094469, -0.21421503, -0.159798  , -0.3393636 ,
         0.188704  , -0.03698638,  0.293989  ,  0.04746273, -0.60596937,
         0.30029285,  0.05095096, -0.12440037, -0.48934975, -0.90608335,
        -0.301625  , -0.25608087,  0.7153497 ,  0.01309137, -0.11807969,
         0.51476234,  0.33734   ,  0.32054278, -0.6

In [49]:
pd.DataFrame(word2vec_embedding)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.423787,0.363309,0.238882,-0.109597,0.213512,-0.152748,0.284102,0.229761,-0.612819,0.427475,...,0.132401,-0.016177,0.153472,-0.747417,0.463523,0.742617,0.249259,-0.038514,-0.049952,0.404782
1,-0.301654,0.466265,0.127860,-0.065042,0.115538,-0.364807,0.469811,0.440320,-0.713322,0.327348,...,0.395621,0.060512,0.103767,-0.444066,0.686454,0.687464,0.109389,-0.188821,-0.005369,0.398273
2,-0.415406,0.689065,0.476271,-0.269254,0.112506,-0.642897,0.138813,0.701064,-0.266966,0.036736,...,0.409588,-0.072331,0.447529,-0.584873,0.724400,0.629442,0.202655,-0.095334,-0.075704,0.264306
3,-0.319704,0.614325,0.433190,-0.178630,0.096009,-0.674789,0.196783,0.683758,-0.184409,-0.032366,...,0.434676,-0.074061,0.336459,-0.461889,0.686309,0.503168,0.150200,-0.145116,-0.024100,0.252631
4,-0.293201,0.463361,0.226370,-0.180645,0.064854,-0.451840,0.269335,0.511820,-0.497319,0.273473,...,0.290775,-0.043458,0.202672,-0.577147,0.653121,0.683651,0.101367,-0.140068,-0.088214,0.487234
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4801,-0.340861,0.943209,0.461615,-0.333557,-0.054509,-0.998336,0.282703,0.986762,-0.225186,-0.163681,...,0.775851,-0.027706,0.451668,-0.265325,1.040259,0.517976,0.127156,-0.270243,-0.014221,0.112699
4802,-0.103024,0.452731,-0.042578,0.320017,0.138242,-0.380190,0.858827,0.442100,-0.873571,-0.001158,...,0.671087,0.293131,-0.130631,0.308788,0.670534,0.529656,-0.055641,-0.338552,0.250150,0.005599
4803,0.087888,0.345841,0.002998,0.051916,-0.054904,-0.687454,0.606868,0.749988,-0.601597,-0.104285,...,0.547285,0.059955,-0.046741,-0.034317,0.784435,0.466175,-0.132637,-0.370894,0.000223,0.506600
4804,0.019944,0.498186,0.088873,-0.018358,-0.079275,-0.729841,0.548237,0.782783,-0.503548,-0.018225,...,0.683595,0.054030,0.007936,0.067683,0.863885,0.450144,-0.213596,-0.283317,0.036585,0.429762


In [50]:
word_2_vec_similar=cosine_similarity(word2vec_embedding)

In [51]:
word_2_vec_similar

array([[0.99999994, 0.9282535 , 0.86116254, ..., 0.58088267, 0.51714045,
        0.43365455],
       [0.9282535 , 1.        , 0.88401705, ..., 0.8159233 , 0.78648174,
        0.7152968 ],
       [0.86116254, 0.88401705, 1.0000001 , ..., 0.712853  , 0.7068017 ,
        0.65530634],
       ...,
       [0.58088267, 0.8159233 , 0.712853  , ..., 0.99999994, 0.96560186,
        0.96360576],
       [0.51714045, 0.78648174, 0.7068017 , ..., 0.96560186, 0.9999998 ,
        0.985912  ],
       [0.43365455, 0.7152968 , 0.65530634, ..., 0.96360576, 0.985912  ,
        1.0000001 ]], dtype=float32)

In [52]:
#Recommender using word2vec
def recommender_word2vec(movie_title):
    index=movies[movies["title"]==movie_title].index[0]
    similarity=list(enumerate(word_2_vec_similar[index]))
    similar_5=sorted(similarity,reverse=True,key=lambda x:x[1])[1:6]
    for data in similar_5:
        print(movies['title'][data[0]])

In [53]:
recommender_tfidf('Batman')

Batman
Superman
Green Lantern
Batman Begins
Batman & Robin


### Hence getting best results using CountVectorizer embedding

In [54]:
movies.to_csv('movies.csv',index=False)

In [55]:
with open("similarity.pkl","wb") as file:
    pickle.dump(cv_similar,file)

In [56]:
title="Avatar"
index=movies[movies["title"]==title].index[0]
similarity=list(enumerate(cv_similar[index]))
similar_5=sorted(similarity,reverse=True,key=lambda x:x[1])[0:6]
movie_name=[]
movie_id=[]
for data in similar_5:

    link=json.loads(requests.get(r"https://api.themoviedb.org/3/movie/"+str(movies['id'][data[0]])+r"?api_key=7ab54985896c47067dda1312d9b6add1").text)['poster_path']
    complete_link="https://image.tmdb.org/t/p/original/"+link
    print(complete_link)

https://image.tmdb.org/t/p/original//jRXYjXNq0Cs2TcJjLkki24MLp7u.jpg
https://image.tmdb.org/t/p/original//wuF8aapqbDPFkoWbwI5LWfDXpVq.jpg
https://image.tmdb.org/t/p/original//jCyJN1vj8jqJJ0vNw4hDH2KlySO.jpg
https://image.tmdb.org/t/p/original//p0BPQGSPoSa8Ml0DAf2mB2kCU0R.jpg
https://image.tmdb.org/t/p/original//2NCcAZ3M3F0FxENYmammBknwpVn.jpg
https://image.tmdb.org/t/p/original//qsUCoeTwCNIrvDrUiWlTCgyRFZy.jpg


In [57]:
movies[movies['title'].str.contains("Batman")]

,id,title,overview,genres,keywords,cast,crew,tags
9,209112,Batman v Superman: Dawn of Justice,Fearing the actions of a god-like Super Hero l...,Action Adventure Fantasy,dccomics vigilante superhero basedoncomicbook ...,BenAffleck HenryCavill GalGadot,TroySizemore,Fearing the actions of a god-like Super Hero l...
119,272,Batman Begins,"Driven by tragedy, billionaire Bruce Wayne ded...",Action Crime Drama,himalaya martialarts dccomics crimefighter sec...,ChristianBale MichaelCaine LiamNeeson,ChristopherNolan,"Driven by tragedy, billionaire Bruce Wayne ded..."
210,415,Batman & Robin,Along with crime-fighting partner Robin and ne...,Action Crime Fantasy,doublelife dccomics dualidentity crimefighter ...,GeorgeClooney ChrisO'Donnell ArnoldSchwarzenegger,JoelSchumacher,Along with crime-fighting partner Robin and ne...
299,414,Batman Forever,The Dark Knight of Gotham City confronts a das...,Action Crime Fantasy,riddle dccomics rose gothamcity partner superh...,ValKilmer TommyLeeJones JimCarrey,JoelSchumacher,The Dark Knight of Gotham City confronts a das...
428,364,Batman Returns,"Having defeated the Joker, Batman now faces th...",Action Fantasy,holiday corruption doublelife dccomics crimefi...,MichaelKeaton DannyDeVito MichellePfeiffer,TimBurton,"Having defeated the Joker, Batman now faces th..."
1362,268,Batman,The Dark Knight of Gotham City begins his war ...,Fantasy Action,doublelife dccomics dualidentity chemical crim...,JackNicholson MichaelKeaton KimBasinger,RogerPratt,The Dark Knight of Gotham City begins his war ...
1363,268,Batman,The Dark Knight of Gotham City begins his war ...,Fantasy Action,doublelife dccomics dualidentity chemical crim...,AdamWest BurtWard LeeMeriwether,LeslieH.Martinson,The Dark Knight of Gotham City begins his war ...
1364,2661,Batman,The Dynamic Duo faces four super-villains who ...,Family Adventure Comedy ScienceFiction Crime,submarine dccomics missile sharkattack rescue ...,JackNicholson MichaelKeaton KimBasinger,RogerPratt,The Dynamic Duo faces four super-villains who ...
1365,2661,Batman,The Dynamic Duo faces four super-villains who ...,Family Adventure Comedy ScienceFiction Crime,submarine dccomics missile sharkattack rescue ...,AdamWest BurtWard LeeMeriwether,LeslieH.Martinson,The Dynamic Duo faces four super-villains who ...
3861,142061,"Batman: The Dark Knight Returns, Part 2",Batman has stopped the reign of terror that Th...,Action Animation,dccomics future joker robin basedongraphicnove...,PeterWeller ArielWinter DavidSelby,JayOliva,Batman has stopped the reign of terror that Th...


In [58]:
"batman & robin".title()

'Batman & Robin'

In [59]:
movies

,id,title,overview,genres,keywords,cast,crew,tags
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,SamWorthington ZoeSaldana SigourneyWeaver,JamesCameron,"In the 22nd century, a paraplegic Marine is di..."
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drugabuse exoticisland eastindiatradingc...,JohnnyDepp OrlandoBloom KeiraKnightley,DariuszWolski,"Captain Barbossa, long believed to be dead, ha..."
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy basedonnovel secretagent sequel mi6 britis...,DanielCraig ChristophWaltz LéaSeydoux,SamMendes,A cryptic message from Bond’s past sends him o...
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dccomics crimefighter terrorist secretidentity...,ChristianBale MichaelCaine GaryOldman,ChristopherNolan,Following the death of District Attorney Harve...
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure ScienceFiction,basedonnovel mars medallion spacetravel prince...,TaylorKitsch LynnCollins SamanthaMorton,AndrewStanton,"John Carter is a war-weary, former military ca..."
...,...,...,...,...,...,...,...,...
4804,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,Action Crime Thriller,unitedstates–mexicobarrier legs arms paperknif...,CarlosGallardo JaimedeHoyos PeterMarquardt,RobertRodriguez,El Mariachi just wants to play his guitar and ...
4805,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,Comedy Romance,,EdwardBurns KerryBishé MarshaDietlein,EdwardBurns,A newlywed couple's honeymoon is upended by th...
4806,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TVMovie,date loveatfirstsight narration investigation ...,EricMabius KristinBooth CrystalLowe,AdamSliwinski,"""Signed, Sealed, Delivered"" introduces a dedic..."
4807,126186,Shanghai Calling,When ambitious New York attorney Sam is sent t...,,,DanielHenney ElizaCoupe BillPaxton,DanielHsia,When ambitious New York attorney Sam is sent t...


In [60]:
def genre(data):
    if data:
        return data.split()[0].lower()
   

movies['new_genre']=movies['genres'].apply(genre)

In [61]:
action=movies.groupby('new_genre').get_group("action")[["id","title"]][:8]
animation=movies.groupby('new_genre').get_group("animation")[["id","title"]][:8]
SciFi=movies.groupby('new_genre').get_group("sciencefiction")[["id","title"]][:8]
romance=movies.groupby('new_genre').get_group("romance")[["id","title"]][:8]

In [62]:
new_movies=movies.merge(movies_data,on='title')

In [63]:
new_movies.to_csv('new_movies.csv',index=False)

### Adding vote_average feature in movies df.

In [64]:
movies=movies.merge(movies_data[["id","vote_average"]],on='id')
movies.dropna(inplace=True)
movies.to_csv("movies.csv",index=False)

In [65]:
movies

,id,title,overview,genres,keywords,cast,crew,tags,new_genre,vote_average
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,SamWorthington ZoeSaldana SigourneyWeaver,JamesCameron,"In the 22nd century, a paraplegic Marine is di...",action,7.2
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drugabuse exoticisland eastindiatradingc...,JohnnyDepp OrlandoBloom KeiraKnightley,DariuszWolski,"Captain Barbossa, long believed to be dead, ha...",adventure,6.9
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy basedonnovel secretagent sequel mi6 britis...,DanielCraig ChristophWaltz LéaSeydoux,SamMendes,A cryptic message from Bond’s past sends him o...,action,6.3
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dccomics crimefighter terrorist secretidentity...,ChristianBale MichaelCaine GaryOldman,ChristopherNolan,Following the death of District Attorney Harve...,action,7.6
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure ScienceFiction,basedonnovel mars medallion spacetravel prince...,TaylorKitsch LynnCollins SamanthaMorton,AndrewStanton,"John Carter is a war-weary, former military ca...",action,6.1
...,...,...,...,...,...,...,...,...,...,...
4800,67238,Cavite,"Adam, a security guard, travels from Californi...",Foreign Thriller,,,NeillDelaLlana,"Adam, a security guard, travels from Californi...",foreign,7.5
4801,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,Action Crime Thriller,unitedstates–mexicobarrier legs arms paperknif...,CarlosGallardo JaimedeHoyos PeterMarquardt,RobertRodriguez,El Mariachi just wants to play his guitar and ...,action,6.6
4802,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,Comedy Romance,,EdwardBurns KerryBishé MarshaDietlein,EdwardBurns,A newlywed couple's honeymoon is upended by th...,comedy,5.9
4803,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TVMovie,date loveatfirstsight narration investigation ...,EricMabius KristinBooth CrystalLowe,AdamSliwinski,"""Signed, Sealed, Delivered"" introduces a dedic...",comedy,7.0


In [66]:
new_movies

,id_x,title,overview_x,genres_x,keywords_x,cast,crew,tags,new_genre,budget,...,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,vote_average,vote_count
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,SamWorthington ZoeSaldana SigourneyWeaver,JamesCameron,"In the 22nd century, a paraplegic Marine is di...",action,237000000,...,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,7.2,11800
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drugabuse exoticisland eastindiatradingc...,JohnnyDepp OrlandoBloom KeiraKnightley,DariuszWolski,"Captain Barbossa, long believed to be dead, ha...",adventure,300000000,...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",6.9,4500
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy basedonnovel secretagent sequel mi6 britis...,DanielCraig ChristophWaltz LéaSeydoux,SamMendes,A cryptic message from Bond’s past sends him o...,action,245000000,...,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,6.3,4466
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dccomics crimefighter terrorist secretidentity...,ChristianBale MichaelCaine GaryOldman,ChristopherNolan,Following the death of District Attorney Harve...,action,250000000,...,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,7.6,9106
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure ScienceFiction,basedonnovel mars medallion spacetravel prince...,TaylorKitsch LynnCollins SamanthaMorton,AndrewStanton,"John Carter is a war-weary, former military ca...",action,260000000,...,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4813,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,Action Crime Thriller,unitedstates–mexicobarrier legs arms paperknif...,CarlosGallardo JaimedeHoyos PeterMarquardt,RobertRodriguez,El Mariachi just wants to play his guitar and ...,action,220000,...,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",6.6,238
4814,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,Comedy Romance,,EdwardBurns KerryBishé MarshaDietlein,EdwardBurns,A newlywed couple's honeymoon is upended by th...,comedy,9000,...,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,5.9,5
4815,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TVMovie,date loveatfirstsight narration investigation ...,EricMabius KristinBooth CrystalLowe,AdamSliwinski,"""Sign

In [ ]:
title="Avatar"
if title.lower() in [text.lower() for text in  movies["title"].to_list()]:
    index=movies[movies["title"]==title.title()].index[0]

In [74]:
movies

,id,title,overview,genres,keywords,cast,crew,tags,new_genre,vote_average
0,19995,Avatar,"In the 22nd century, a paraplegic Marine is di...",Action Adventure Fantasy ScienceFiction,cultureclash future spacewar spacecolony socie...,SamWorthington ZoeSaldana SigourneyWeaver,JamesCameron,"In the 22nd century, a paraplegic Marine is di...",action,7.2
1,285,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",Adventure Fantasy Action,ocean drugabuse exoticisland eastindiatradingc...,JohnnyDepp OrlandoBloom KeiraKnightley,DariuszWolski,"Captain Barbossa, long believed to be dead, ha...",adventure,6.9
2,206647,Spectre,A cryptic message from Bond’s past sends him o...,Action Adventure Crime,spy basedonnovel secretagent sequel mi6 britis...,DanielCraig ChristophWaltz LéaSeydoux,SamMendes,A cryptic message from Bond’s past sends him o...,action,6.3
3,49026,The Dark Knight Rises,Following the death of District Attorney Harve...,Action Crime Drama Thriller,dccomics crimefighter terrorist secretidentity...,ChristianBale MichaelCaine GaryOldman,ChristopherNolan,Following the death of District Attorney Harve...,action,7.6
4,49529,John Carter,"John Carter is a war-weary, former military ca...",Action Adventure ScienceFiction,basedonnovel mars medallion spacetravel prince...,TaylorKitsch LynnCollins SamanthaMorton,AndrewStanton,"John Carter is a war-weary, former military ca...",action,6.1
...,...,...,...,...,...,...,...,...,...,...
4800,67238,Cavite,"Adam, a security guard, travels from Californi...",Foreign Thriller,,,NeillDelaLlana,"Adam, a security guard, travels from Californi...",foreign,7.5
4801,9367,El Mariachi,El Mariachi just wants to play his guitar and ...,Action Crime Thriller,unitedstates–mexicobarrier legs arms paperknif...,CarlosGallardo JaimedeHoyos PeterMarquardt,RobertRodriguez,El Mariachi just wants to play his guitar and ...,action,6.6
4802,72766,Newlyweds,A newlywed couple's honeymoon is upended by th...,Comedy Romance,,EdwardBurns KerryBishé MarshaDietlein,EdwardBurns,A newlywed couple's honeymoon is upended by th...,comedy,5.9
4803,231617,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",Comedy Drama Romance TVMovie,date loveatfirstsight narration investigation ...,EricMabius KristinBooth CrystalLowe,AdamSliwinski,"""Signed, Sealed, Delivered"" introduces a dedic...",comedy,7.0


In [90]:
bool(len(movies[movies['title'].apply(lambda x: x.lower()).str.contains('xxxxxxxxxxxxxxx')]))

False

In [95]:
title='Batman'
bool (len(movies[movies['title'].apply(lambda x: x.lower()).str.contains(title.lower())]))

True

In [98]:
index=movies[movies['title'].apply(lambda x: x.lower()).str.contains(title.lower())].index[0]
index

9

In [99]:
movies.duplicated().sum()

0